## pytorchの演算の扱い

In [6]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [7]:
#行を連結
# By default, it concatenates along the first axis (concatenates rows)
x_1 = torch.randn(2, 5)
y_1 = torch.randn(3, 5)
z_1 = torch.cat([x_1, y_1])
print(z_1)

#列を連結
# Concatenate columns:
x_2 = torch.randn(2, 3)
y_2 = torch.randn(2, 5)
# second arg specifies which axis to concat along
z_2 = torch.cat([x_2, y_2], 1)
print(z_2)


-2.9718  1.7070 -0.4305 -2.2820  0.5237
 0.0004 -1.2039  3.5283  0.4434  0.5848
 0.8407  0.5510  0.3863  0.9124 -0.8410
 1.2282 -1.8661  1.4146 -1.8781 -0.4674
-0.7576  0.4215 -0.4827 -1.1198  0.3056
[torch.FloatTensor of size 5x5]


 1.0386  0.5206 -0.5006 -1.9441 -0.9596  0.5489 -0.9901 -0.3826
 1.2182  0.2117 -1.0613  1.5037  1.8267  0.5561  1.6445  0.4973
[torch.FloatTensor of size 2x8]



In [8]:
#.view：reshape関数
x = torch.randn(2, 3, 4)
print(x)

#2*12にreshape
print(x.view(2, 12))  # Reshape to 2 rows, 12 columns

#上記と同じ。最終層を一つ減らす。サイズは推論される
# Same as above.  If one of the dimensions is -1, its size can be inferred
print(x.view(2, -1))


(0 ,.,.) = 
 -1.5067  1.7661 -0.3569 -0.1713
  0.4068 -0.4284 -1.1299  1.4274
 -1.4027  1.4825 -1.1559  1.6190

(1 ,.,.) = 
  0.9581  0.7747  0.1940  0.1687
  0.3061  1.0743 -1.0327  1.0930
  0.7769 -1.3128  0.7099  0.9944
[torch.FloatTensor of size 2x3x4]



Columns 0 to 9 
-1.5067  1.7661 -0.3569 -0.1713  0.4068 -0.4284 -1.1299  1.4274 -1.4027  1.4825
 0.9581  0.7747  0.1940  0.1687  0.3061  1.0743 -1.0327  1.0930  0.7769 -1.3128

Columns 10 to 11 
-1.1559  1.6190
 0.7099  0.9944
[torch.FloatTensor of size 2x12]



Columns 0 to 9 
-1.5067  1.7661 -0.3569 -0.1713  0.4068 -0.4284 -1.1299  1.4274 -1.4027  1.4825
 0.9581  0.7747  0.1940  0.1687  0.3061  1.0743 -1.0327  1.0930  0.7769 -1.3128

Columns 10 to 11 
-1.1559  1.6190
 0.7099  0.9944
[torch.FloatTensor of size 2x12]



## NLP導入

In [9]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

#出てくる単語にidを割り振る
# word_to_ix maps each word in the vocab to a unique integer, which will be its
# index into the Bag of words vector
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            #長さは1ずつ伸びるのでこれをidとして利用
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

{'en': 3, 'No': 9, 'buena': 14, 'it': 7, 'at': 22, 'sea': 12, 'cafeteria': 5, 'Yo': 23, 'la': 4, 'to': 8, 'creo': 10, 'is': 16, 'a': 18, 'good': 19, 'get': 20, 'idea': 15, 'que': 11, 'not': 17, 'me': 0, 'on': 25, 'gusta': 1, 'lost': 21, 'Give': 6, 'una': 13, 'si': 24, 'comer': 2}


In [10]:
class BoWClassifier(nn.Module):  # inheriting from nn.Module!

    def __init__(self, num_labels, vocab_size):
        # calls the init function of nn.Module.  Dont get confused by syntax,
        # just always do it in an nn.Module
        super(BoWClassifier, self).__init__()

        # Define the parameters that you will need.  In this case, we need A and b,
        # the parameters of the affine mapping.
        # Torch defines nn.Linear(), which provides the affine map.
        # Make sure you understand why the input dimension is vocab_size
        # and the output is num_labels!
        self.linear = nn.Linear(vocab_size, num_labels)

        # NOTE! The non-linearity log softmax does not have parameters! So we don't need
        # to worry about that here

    def forward(self, bow_vec):
        # Pass the input through the linear layer,
        # then pass that through log_softmax.
        # Many non-linearities and other functions are in torch.nn.functional
        return F.log_softmax(self.linear(bow_vec))


def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)


def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])


model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

In [11]:
# the model knows its parameters.  The first output below is A, the second is b.
# Whenever you assign a component to a class variable in the __init__ function
# of a module, which was done with the line
# self.linear = nn.Linear(...)
# Then through some Python magic from the Pytorch devs, your module
# (in this case, BoWClassifier) will store knowledge of the nn.Linear's parameters
for param in model.parameters():
    print(param)

# To run the model, pass in a BoW vector, but wrapped in an autograd.Variable
sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_ix)
log_probs = model(autograd.Variable(bow_vector))
print(log_probs)

Parameter containing:

Columns 0 to 9 
 0.0130 -0.1186  0.0753 -0.0825 -0.0724 -0.1404  0.0732  0.1111  0.1313 -0.0343
 0.1603 -0.1584 -0.0810  0.1582 -0.0832 -0.1492 -0.1451  0.0097 -0.1885 -0.1633

Columns 10 to 19 
-0.1889 -0.1827  0.0981  0.0486  0.1917  0.0630  0.0973 -0.0790 -0.0861 -0.0211
 0.0701  0.1635 -0.1131  0.1610 -0.0920 -0.0789 -0.0033  0.0331 -0.1752  0.0259

Columns 20 to 25 
 0.1135 -0.1090 -0.1556 -0.1673 -0.0204 -0.0121
 0.0291  0.0447 -0.1386  0.1791  0.0350 -0.0938
[torch.FloatTensor of size 2x26]

Parameter containing:
 0.0784
-0.1055
[torch.FloatTensor of size 2]

Variable containing:
-0.6874 -0.6990
[torch.FloatTensor of size 1x2]



In [13]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

# Run on test data before we train, just to see a before-and-after
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)

# Print the matrix column corresponding to "creo"
print(next(model.parameters())[:, word_to_ix["creo"]])

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Usually you want to pass over the training data several times.
# 100 is much bigger than on a real data set, but real datasets have more than
# two instances.  Usually, somewhere between 5 and 30 epochs is reasonable.
for epoch in range(100):
    for instance, label in data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Make our BOW vector and also we must wrap the target in a
        # Variable as an integer. For example, if the target is SPANISH, then
        # we wrap the integer 0. The loss function then knows that the 0th
        # element of the log probabilities is the log probability
        # corresponding to SPANISH
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
        target = autograd.Variable(make_target(label, label_to_ix))

        # Step 3. Run our forward pass.
        log_probs = model(bow_vec)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)

# Index corresponding to Spanish goes up, English goes down!
print(next(model.parameters())[:, word_to_ix["creo"]])

Variable containing:
-1.1873 -0.3639
[torch.FloatTensor of size 1x2]

Variable containing:
-0.6133 -0.7799
[torch.FloatTensor of size 1x2]

Variable containing:
-0.1889
 0.0701
[torch.FloatTensor of size 2]

Variable containing:
-0.3196 -1.2963
[torch.FloatTensor of size 1x2]

Variable containing:
-2.9951 -0.0513
[torch.FloatTensor of size 1x2]

Variable containing:
 0.2484
-0.3672
[torch.FloatTensor of size 2]

